In [2]:
# Install Qiskit in the current environment
!pip install qiskit qiskit-aer matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00


In [3]:
# Task 1: Bernstein-Vazirani with new secret string s
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def bv_oracle(qc, inputs, ancilla, s):
    """Oracle for f(x) = s · x (no constant)."""
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

def bernstein_vazirani_circuit(s):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)          # Initialize ancilla to |1>
    qc.h(qreg)             # Apply H to all qubits
    bv_oracle(qc, inputs, ancilla, s)
    for q in inputs:
        qc.h(q)            # Apply H to input qubits
    qc.measure(inputs, creg)
    return qc

def run_bv(qc, shots=1024):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    job = sim.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    print('Counts:', counts)
    fig = plot_histogram(counts)
    plt.show()
    most = max(counts, key=counts.get)
    print('Most frequent measured bitstring (input register):', most)
    return most

# Change the secret string here
if __name__ == '__main__':
    s = '1100'  # New secret string
    print('Secret string s =', s)
    qc = bernstein_vazirani_circuit(s)
    print(qc.draw(fold=-1))
    measured = run_bv(qc)
    if measured == s:
        print('✅ Successfully recovered secret string s')
    else:
        print('⚠️ Measured string differs from s (noise or error).')


Secret string s = 1100
     ┌───┐          ┌───┐             ┌─┐   
q_0: ┤ H ├───────■──┤ H ├─────────────┤M├───
     ├───┤       │  └───┘        ┌───┐└╥┘┌─┐
q_1: ┤ H ├───────┼────────────■──┤ H ├─╫─┤M├
     ├───┤┌───┐  │   ┌─┐      │  └───┘ ║ └╥┘
q_2: ┤ H ├┤ H ├──┼───┤M├──────┼────────╫──╫─
     ├───┤├───┤  │   └╥┘ ┌─┐  │        ║  ║ 
q_3: ┤ H ├┤ H ├──┼────╫──┤M├──┼────────╫──╫─
     ├───┤├───┤┌─┴─┐  ║  └╥┘┌─┴─┐      ║  ║ 
q_4: ┤ X ├┤ H ├┤ X ├──╫───╫─┤ X ├──────╫──╫─
     └───┘└───┘└───┘  ║   ║ └───┘      ║  ║ 
c: 4/═════════════════╩═══╩════════════╩══╩═
                      2   3            0  1 
Counts: {'0011': 1024}
Most frequent measured bitstring (input register): 0011
⚠️ Measured string differs from s (noise or error).


In [4]:
# Task 2: Bernstein-Vazirani with constant bit b
def bv_oracle_with_b(qc, inputs, ancilla, s, b):
    """Oracle for f(x) = s · x ⊕ b."""
    # Apply s·x part
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)
    # Apply constant b
    if b == '1':
        qc.x(ancilla)

def bernstein_vazirani_circuit_with_b(s, b='0'):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)
    bv_oracle_with_b(qc, inputs, ancilla, s, b)
    for q in inputs:
        qc.h(q)
    qc.measure(inputs, creg)
    return qc

if __name__ == '__main__':
    s = '1011'
    b = '1'
    print('Secret string s =', s, ', constant b =', b)
    qc = bernstein_vazirani_circuit_with_b(s, b)
    print(qc.draw(fold=-1))
    measured = run_bv(qc)


Secret string s = 1011 , constant b = 1
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ ├───┤ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫─┤ X ├─╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║ └───┘ ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
Counts: {'1101': 1024}
Most frequent measured bitstring (input register): 1101


In [14]:
# Task 4: Run BV with noise using AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit import transpile
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Define a simple noise model
noise_model = NoiseModel()

# 1% depolarizing error on single-qubit gates (X, H)
single_qubit_error = depolarizing_error(0.01, 1)
noise_model.add_all_qubit_quantum_error(single_qubit_error, ['x', 'h'])

# 2% depolarizing error on two-qubit gates (CX)
two_qubit_error = depolarizing_error(0.02, 2)
noise_model.add_all_qubit_quantum_error(two_qubit_error, ['cx'])

# Use AerSimulator with the noise model
from qiskit_aer import AerSimulator
sim = AerSimulator(noise_model=noise_model)

# Reuse the Bernstein–Vazirani circuit (from Task 3)
s = '1011'
b = '0'
qc = bernstein_vazirani_circuit_with_b(s, b)  # make sure function is already defined

# Transpile circuit for the noisy simulator
tqc = transpile(qc, sim)

# Run simulation
job = sim.run(tqc, shots=1024)
result = job.result()
counts_noisy = result.get_counts()

# Show histogram and results
print("Counts with noise:", counts_noisy)
plot_histogram(counts_noisy)
plt.show()

# Most frequent bitstring under noise
most_noisy = max(counts_noisy, key=counts_noisy.get)
print("Most frequent measured bitstring under noise:", most_noisy)

# Check if secret string is still correctly recovered
if most_noisy == s:
    print("✅ Secret string recovered even with noise")
else:
    print("⚠️ Noise affected the measurement")


Counts with noise: {'0100': 1, '0001': 4, '0000': 19, '1000': 2, '1001': 18, '1100': 16, '0101': 33, '1101': 931}
Most frequent measured bitstring under noise: 1101
⚠️ Noise affected the measurement


In [15]:
# ===============================
# Task 0: Install dependencies
# ===============================
!pip install qiskit qiskit-aer matplotlib

# ===============================
# Task 1: Import libraries
# ===============================
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from qiskit_aer.noise import NoiseModel, depolarizing_error

# ===============================
# Task 2: Define Bernstein-Vazirani Circuit
# ===============================
def bv_oracle_with_b(qc, inputs, ancilla, s, b):
    """Implements oracle for f(x) = s·x ⊕ b"""
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)
    if b == '1':
        qc.x(ancilla)

def bernstein_vazirani_circuit_with_b(s, b='0'):
    """Constructs Bernstein-Vazirani circuit"""
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n

    # Step 1: Initialize ancilla to |1> and apply Hadamard to all qubits
    qc.x(ancilla)
    qc.h(qreg)

    # Step 2: Apply oracle
    bv_oracle_with_b(qc, inputs, ancilla, s, b)

    # Step 3: Apply Hadamard to input qubits
    for q in inputs:
        qc.h(q)

    # Step 4: Measure input qubits
    qc.measure(inputs, creg)
    return qc

# ===============================
# Task 1: Test different secret strings (Task 1)
# ===============================
s = '1011'   # Secret string
b = '0'      # Constant bit
qc = bernstein_vazirani_circuit_with_b(s, b)

print("Quantum Circuit:")
print(qc.draw(fold=-1))

# ===============================
# Task 3: Run on local simulator
# ===============================
sim = AerSimulator()
tqc = transpile(qc, sim)
job = sim.run(tqc, shots=1024)
result = job.result()
counts = result.get_counts()

print("Counts (simulator):", counts)
plot_histogram(counts)
plt.show()

most = max(counts, key=counts.get)
print("Most frequent measured bitstring:", most)

if most == s:
    print("✅ Secret string recovered successfully")
else:
    print("⚠️ Secret string differs (unexpected)")

# ===============================
# Task 4: Add noise and analyze robustness
# ===============================
# Define noise model
noise_model = NoiseModel()
single_qubit_error = depolarizing_error(0.01, 1)
two_qubit_error = depolarizing_error(0.02, 2)
noise_model.add_all_qubit_quantum_error(single_qubit_error, ['x', 'h'])
noise_model.add_all_qubit_quantum_error(two_qubit_error, ['cx'])

# Run circuit on noisy simulator
noisy_sim = AerSimulator(noise_model=noise_model)
tqc_noisy = transpile(qc, noisy_sim)
job_noisy = noisy_sim.run(tqc_noisy, shots=1024)
result_noisy = job_noisy.result()
counts_noisy = result_noisy.get_counts()

print("Counts with noise:", counts_noisy)
plot_histogram(counts_noisy)
plt.show()

most_noisy = max(counts_noisy, key=counts_noisy.get)
print("Most frequent bitstring with noise:", most_noisy)
if most_noisy == s:
    print("✅ Secret string recovered even with noise")
else:
    print("⚠️ Noise affected the measurement")

# ===============================
# Task 5: Notebook-style explanations
# ===============================
# You can add Markdown cells in your notebook explaining:
# 1. Introduction to Bernstein–Vazirani Algorithm
# 2. Purpose of the ancilla qubit
# 3. How the oracle works (s·x ⊕ b)
# 4. Role of Hadamard gates before and after oracle
# 5. Measurement of input qubits reveals the secret string
# 6. Noise analysis and its effect on quantum computation
# 7. Visualizations (histograms) for both noiseless and noisy simulations

# You can also add Markdown diagrams using LaTeX to explain:
# - Quantum states after each step
# - Effect of the oracle
# - How the Hadamard transforms reveal the secret string


Quantum Circuit:
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
Counts (simulator): {'1101': 1024}
Most frequent measured bitstring: 1101
⚠️ Secret string differs (unexpected)
Counts with noise: {'0100': 2, '1100': 17, '1001': 17, '0101': 21, '0000': 12, '0001': 10, '1101': 945}
Most frequent bitstring with noise: 1101
⚠️ Noise affected the measurement
